# DSC212 — Modularity on the Karate Club Graph

This notebook implements recursive spectral modularity partitioning on Zachary's Karate Club graph. It follows the assignment specification: compute modularity matrix, perform spectral bipartitioning, recursively split communities while the leading eigenvalue is positive, visualize splits, compute node metrics, and plot metric evolution.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import defaultdict
import pandas as pd

# For reproducible layout
np.random.seed(0)


## 1. Load the Karate Club graph and compute basic quantities

We'll load the graph using NetworkX's built-in Karate Club dataset and compute adjacency, degree, and m (number of edges).

In [ ]:
G = nx.karate_club_graph()

n = G.number_of_nodes()
m = G.number_of_edges()
A = nx.to_numpy_array(G, dtype=int)
ks = np.array([d for _, d in G.degree()], dtype=float)

n, m


## 2. Build the modularity matrix $B = A - \frac{kk^T}{2m}$

In [ ]:
def modularity_matrix(A, k, m):
    k = k.reshape((-1,1))
    return A - (k @ k.T) / (2*m)

B = modularity_matrix(A, ks, m)
B.shape


## 3. Spectral bipartition helper functions

We implement functions to compute the leading eigenpair of a (restricted) modularity matrix and to convert the eigenvector into a binary partition using the sign rule.

In [ ]:
def leading_eigenpair(M):
    # Use eigh (symmetric) for numerical stability
    vals, vecs = np.linalg.eigh(M)
    idx = np.argsort(vals)[::-1]
    vals = vals[idx]
    vecs = vecs[:, idx]
    return vals[0], vecs[:, 0]

# Test on full B
lambda1, u1 = leading_eigenpair(B)
lambda1


In [ ]:
def partition_by_leading_eig(B, nodes_index):
    # B: modularity matrix (full global B)
    # nodes_index: list or array of node indices to restrict to
    if len(nodes_index) == 0:
        return None
    M = B[np.ix_(nodes_index, nodes_index)]
    lam, u = leading_eigenpair(M)
    s = np.ones(len(nodes_index), dtype=int)
    s[u <= 0] = -1
    return lam, s, u

# Quick demo for full node set
nodes_idx = list(range(n))
lam_full, s_full, u_full = partition_by_leading_eig(B, nodes_idx)
lam_full


## 4. Recursive bisection algorithm

We recursively split communities while the leading eigenvalue of the restricted modularity matrix is positive. We store community assignments after each split for visualization and metric tracking.

In [ ]:
def recursive_bisect(B, initial_nodes=None):
    if initial_nodes is None:
        initial_nodes = list(range(B.shape[0]))
    communities = [initial_nodes]
    snapshots = [list(initial_nodes)]
    queue = [list(initial_nodes)]
    while queue:
        C = queue.pop(0)
        if len(C) <= 1:
            continue
        M = B[np.ix_(C, C)]
        lam, u = leading_eigenpair(M)
        if lam <= 0:
            continue
        s = np.ones(len(C), dtype=int)
        s[u <= 0] = -1
        C_plus = [C[i] for i in range(len(C)) if s[i] == 1]
        C_minus = [C[i] for i in range(len(C)) if s[i] == -1]
        try:
            communities.remove(C)
        except ValueError:
            pass
        communities.append(C_plus)
        communities.append(C_minus)
        queue.append(C_plus)
        queue.append(C_minus)
        snapshots.append([list(c) for c in communities])
    return snapshots

snapshots = recursive_bisect(B)
len(snapshots)


## 5. Visualization of splits

We'll visualize the graph after each recorded split snapshot using a fixed spring layout to keep node positions consistent across plots.

In [ ]:
pos = nx.spring_layout(G, seed=42)

def plot_snapshot(G, snapshot_communities, pos, title=None, savepath=None):
    plt.figure(figsize=(6,6))
    comm_id = {}
    for cid, comm in enumerate(snapshot_communities):
        for node in comm:
            comm_id[node] = cid
    node_colors = [comm_id[i] for i in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=350, cmap=None, node_color=node_colors)
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    nx.draw_networkx_labels(G, pos)
    plt.title(title if title else 'Community snapshot')
    plt.axis('off')
    if savepath:
        plt.savefig(savepath, bbox_inches='tight')
    plt.show()

viz_paths = []
for it, snap in enumerate(snapshots):
    title = f"Snapshot {it}: {len(snap)} communities"
    path = f"/mnt/data/snapshot_{it}.png"
    plot_snapshot(G, snap, pos, title=title, savepath=path)
    viz_paths.append(path)

viz_paths


## 6. Compute centrality and clustering metrics after each split

For each snapshot we compute degree centrality, betweenness centrality, closeness, and clustering coefficient for all nodes and store them for evolution plotting.

In [ ]:
def compute_metrics(G):
    deg = nx.degree_centrality(G)
    btw = nx.betweenness_centrality(G)
    clo = nx.closeness_centrality(G)
    clust = nx.clustering(G)
    nodes_sorted = sorted(G.nodes())
    return {
        'degree': np.array([deg[i] for i in nodes_sorted]),
        'betweenness': np.array([btw[i] for i in nodes_sorted]),
        'closeness': np.array([clo[i] for i in nodes_sorted]),
        'clustering': np.array([clust[i] for i in nodes_sorted])
    }

metrics_history = defaultdict(list)
for it, snap in enumerate(snapshots):
    metrics = compute_metrics(G)
    for k, arr in metrics.items():
        metrics_history[k].append(arr)

metrics_arrays = {k: np.vstack(v) for k, v in metrics_history.items()}
{ k: metrics_arrays[k].shape for k in metrics_arrays }


## 7. Plot metric evolution across iterations

We plot one figure per metric. Each line shows a node's metric value across snapshots (iterations).

In [ ]:
iterations = metrics_arrays['degree'].shape[0]
nodes = sorted(G.nodes())

for metric_name, arr in metrics_arrays.items():
    plt.figure(figsize=(8,5))
    for node_idx in range(len(nodes)):
        plt.plot(range(iterations), arr[:, node_idx])
    plt.xlabel('Iteration (snapshot)')
    plt.ylabel(metric_name)
    plt.title(f'Evolution of {metric_name} across snapshots')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


## 8. Short discussion

Observations (write-up guidance):
- Nodes with high degree centrality at the start often remain influential across splits.
- Betweenness highlights bridge nodes that may shift importance as communities split.
- Clustering coefficient is local and can increase for nodes inside tightly-knit communities after splitting.

You should expand this section in the final report: identify specific node IDs that remain central and relate their metrics to the visual community structure.